<a href="https://colab.research.google.com/github/seonghunYang/big_contest2020/blob/master/EDA7_%EC%83%81%EB%8C%80%EC%A0%84%EC%A0%81%EB%B3%84_%EC%84%B8%EC%9D%B4%EB%B2%84%EB%A9%94%ED%8A%B8%EB%A6%AD%EC%8A%A4_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
path = './gdrive/My Drive/빅콘테스트2020/데이터/2020빅콘테스트 데이터분석분야-퓨쳐스리그_스포츠투아이_제공데이터(.CSV)_시즌별, 시트별 구분/'

In [4]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
team_hitter_2016 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2016.csv', encoding='utf-8-sig')
team_hitter_2017 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2017.csv', encoding='utf-8-sig')
team_hitter_2018 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2018.csv', encoding='utf-8-sig')
team_hitter_2019 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2019.csv', encoding='utf-8-sig')
team_hitter_2020 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2020.csv', encoding='utf-8-sig')
team_hitter_list = [team_hitter_2016, team_hitter_2017, team_hitter_2018, team_hitter_2019, team_hitter_2020]
team_hitter = pd.concat(team_hitter_list)
team_hitter_list.append(team_hitter)

In [6]:
new_columns = "게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타"
new_columns = new_columns.split(',')
for team_h in team_hitter_list:
  team_h.columns = new_columns

In [7]:
team_hitter.head()

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타
0,20160401HHLG0,20160401,LG,HH,0,B,47,42,4,5,9,2,0,1,2,1,1,0,4,0,0,11,0,0,8,0.333333,12,4
1,20160401HHLG0,20160401,HH,LG,0,T,52,46,4,4,13,2,0,0,0,0,3,0,3,0,0,10,1,2,12,0.200000,15,3
2,20160401HTNC0,20160401,NC,HT,0,B,36,30,5,5,9,2,0,2,0,0,1,0,5,0,0,9,1,1,7,0.142857,7,1
3,20160401HTNC0,20160401,HT,NC,0,T,38,34,3,4,8,3,0,1,0,0,0,0,3,0,1,10,1,0,7,0.100000,10,1
4,20160401KTSK0,20160401,SK,KT,0,B,36,36,4,4,10,5,0,1,0,0,0,0,0,0,0,7,1,1,5,0.375000,8,3


In [8]:
for hitter_i in team_hitter_list:
  hitter_i['일자'] = pd.to_datetime(hitter_i['일자'],format='%Y%m%d', errors='raise')
  hitter_i['시즌'] = hitter_i['일자'].dt.year 
  
for hitter_i in team_hitter_list:
  hitter_i['단타'] = hitter_i['안타'] - (hitter_i['2루타'] + hitter_i['3루타'] + hitter_i['홈런'])

In [9]:
def runBaseP(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드']) & (team_hitter['상대팀코드'] == row['상대팀코드']) & (team_hitter['일자'] < row['일자']) ]
    if len(previous) == 0:
        row['출루율'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['안타'] + previous_sum['4구']  + previous_sum['사구']
    down = previous_sum['사구'] + previous_sum['타수'] + previous_sum['4구'] + previous_sum['희비']
    if down == 0:
        row['출루율'] = 0
        return row
    row['출루율'] = up / down
    return row

def longHitP(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드']) & (team_hitter['상대팀코드'] == row['상대팀코드']) & (team_hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['장타력'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['단타'] * 1 + previous_sum['2루타'] * 2 + previous_sum['3루타'] * 3 + previous_sum['홈런'] * 4
    down = previous_sum['타수']
    if down == 0:
        row['장타력'] = 0
        return row
    row['장타력'] = up / down

    return row
def hitP(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드']) & (team_hitter['상대팀코드'] == row['상대팀코드']) & (team_hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['타율'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['안타']
    down = previous_sum['타수']
    if down == 0:
        row['타율'] = 0
        return row
    row['타율'] = up / down
    return row

def createBABIP(row):
  previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드']) & (team_hitter['상대팀코드'] == row['상대팀코드']) & (team_hitter['일자'] < row['일자'])]
  if len(previous) == 0:
      row['BABIP'] = 0
      return row
  previous_sum = previous.sum()
  up = previous_sum['안타'] - previous_sum['홈런']
  down = previous_sum['타수'] - previous_sum['홈런'] - previous_sum['삼진'] + previous_sum['희비']
  if down == 0:
      row['BABIP'] = 0
      return row
  row['BABIP'] = up / down
  return row

def createBBK(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드']) & (team_hitter['상대팀코드'] == row['상대팀코드']) & (team_hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['BB/K'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['4구']
    down = previous_sum['삼진']
    if down == 0:
        row['BB/K'] = 0
        return row
    row['BB/K'] = up / down
    return row

def createK_BB(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드']) & (team_hitter['상대팀코드'] == row['상대팀코드']) & (team_hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['K%'] = 0
        row['BB%'] = 0
        return row
    previous_sum = previous.sum()
    up_k = previous_sum['삼진']
    up_BB = previous_sum['4구'] - previous_sum['고4']
    down = previous_sum['타자'] - previous_sum['고4']
    if down == 0:
        row['K%'] = 0
        row['BB%'] = 0
        return row
    row['K%'] = up_k / down
    row['BB%'] = up_BB / down
    return row

In [10]:
team_hitter = team_hitter.apply(runBaseP,axis=1)
team_hitter = team_hitter.apply(longHitP,axis=1)
team_hitter['ops'] = team_hitter['출루율'] + team_hitter['장타력']
team_hitter = team_hitter.apply(hitP,axis=1)
team_hitter['ISO'] = team_hitter['장타력'] - team_hitter['타율']
team_hitter = team_hitter.apply(createBABIP,axis=1)
team_hitter = team_hitter.apply(createBBK,axis=1)
team_hitter = team_hitter.apply(createK_BB,axis=1)

In [11]:
team_hitter

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타,시즌,단타,출루율,장타력,ops,타율,ISO,BABIP,BB/K,K%,BB%
0,20160401HHLG0,2016-04-01,LG,HH,0,B,47,42,4,5,9,2,0,1,2,1,1,0,4,0,0,11,0,0,8,0.333333,12,4,2016,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,20160401HHLG0,2016-04-01,HH,LG,0,T,52,46,4,4,13,2,0,0,0,0,3,0,3,0,0,10,1,2,12,0.200000,15,3,2016,11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,20160401HTNC0,2016-04-01,NC,HT,0,B,36,30,5,5,9,2,0,2,0,0,1,0,5,0,0,9,1,1,7,0.142857,7,1,2016,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,20160401HTNC0,2016-04-01,HT,NC,0,T,38,34,3,4,8,3,0,1,0,0,0,0,3,0,1,10,1,0,7,0.100000,10,1,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,20160401KTSK0,2016-04-01,SK,KT,0,B,36,36,4,4,10,5,0,1,0,0,0,0,0,0,0,7,1,1,5,0.375000,8,3,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,20200719LTSS0,2020-07-19,LT,SS,0,T,31,29,2,2,6,1,0,1,0,0,1,0,1,0,0,7,3,0,2,0.000000,2,0,2020,4,0.299663,0.314607,0.614270,0.228464,0.086142,0.263636,0.565217,0.154362,0.087248
636,20200719OBHT0,2020-07-19,HT,OB,0,B,38,32,4,4,8,0,1,0,0,0,0,0,6,0,0,5,1,0,7,0.571429,7,4,2020,7,0.340000,0.389513,0.729513,0.265918,0.123596,0.303167,0.681818,0.144737,0.095395
637,20200719OBHT0,2020-07-19,OB,HT,0,T,41,37,8,8,11,3,0,1,2,1,0,0,2,1,2,4,0,0,6,0.363636,11,4,2020,7,0.363934,0.433213,0.797147,0.314079,0.119134,0.360870,0.382979,0.152104,0.058252
638,20200719WOSK0,2020-07-19,SK,WO,0,B,37,30,3,4,6,0,0,0,0,0,0,0,7,0,0,10,0,0,9,0.250000,8,2,2020,6,0.306931,0.365942,0.672873,0.246377,0.119565,0.287081,0.377049,0.200000,0.075410


In [14]:
c_dic = {2020: [0.730, 0.759,	0.938, 1.284,	1.595,	1.959],
         2019: [0.721,	0.752,	0.945,	1.317,	1.653,	2.074],
         2018: [0.726,	0.753,	0.920,	1.242,	1.533,	1.857],
         2017: [0.728,	0.756,	0.927,	1.259,	1.558,	1.901],
         2016: [0.747,	0.774,	0.944,	1.273,	1.569,	1.898]}

c_dic2 = {2020: [0.345,	1.153, 4328, 56881],
         2019: [0.335,	1.242, 6548, 55612],
         2018: [0.351,	1.075, 7994, 56903],
         2017: [0.351,	1.107, 7682, 56881],
         2016: [0.362,	1.097, 8074, 57614]}

In [12]:
def createwOBA(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드'])& (team_hitter['상대팀코드'] == row['상대팀코드']) & (team_hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['wOBA'] = 0
        return row
    previous_sum = previous.sum()
    ct = c_dic[row['시즌']]
    up = (ct[0] * (previous_sum['4구'] - previous_sum['고4']) + ct[1] * previous_sum['사구'] + ct[2] * previous_sum['단타'] + ct[3] * previous_sum['2루타'] + ct[4] * previous_sum['3루타'] + ct[5] * previous_sum['홈런'] )
    down = (previous_sum['타수'] + previous_sum['4구'] - previous_sum['고4'] + previous_sum['사구'] + previous_sum['희비'])
    if down == 0:
        row['wOBA'] = 0
        return row
    row['wOBA'] = up / down
    return row

def createwRAA(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드'])& (team_hitter['상대팀코드'] == row['상대팀코드']) & (team_hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['wRAA'] = 0
        return row
    previous_sum = previous.sum()
    ct = c_dic2[row['시즌']]
    up = row['wOBA'] - ct[0]
    down = ct[1]
    row['wRAA'] = up / down * previous_sum['타자']
    return row

def createwRC(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드'])& (team_hitter['상대팀코드'] == row['상대팀코드']) & (team_hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['wRC'] = 0
        return row
    previous_sum = previous.sum()
    ct = c_dic2[row['시즌']]
    up = ct[2]
    down = ct[3]
    row['wRC'] = (up / down) * previous_sum['타자'] + row['wRAA']
    return row

def createHR(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드'])& (team_hitter['상대팀코드'] == row['상대팀코드']) & (team_hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['HR%'] = 0
        return row
    previous_sum = previous.sum()
    up_hr = previous_sum['홈런']
    down = previous_sum['타자'] - previous_sum['고4']
    if down == 0:
        row['HR%'] = 0
        return row
        
    row['HR%'] = up_hr / down
    return row

In [15]:
team_hitter = team_hitter.apply(createwOBA, axis=1)
team_hitter = team_hitter.apply(createwRAA, axis=1)
team_hitter = team_hitter.apply(createwRC, axis=1)
team_hitter = team_hitter.apply(createHR, axis=1)
team_hitter['GPA'] = (1.8 * team_hitter['출루율'] + team_hitter['장타력']) / 4 

In [16]:
team_hitter

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타,시즌,단타,출루율,장타력,ops,타율,ISO,BABIP,BB/K,K%,BB%,wOBA,wRAA,wRC,HR%,GPA
0,20160401HHLG0,2016-04-01,LG,HH,0,B,47,42,4,5,9,2,0,1,2,1,1,0,4,0,0,11,0,0,8,0.333333,12,4,2016,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,20160401HHLG0,2016-04-01,HH,LG,0,T,52,46,4,4,13,2,0,0,0,0,3,0,3,0,0,10,1,2,12,0.200000,15,3,2016,11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,20160401HTNC0,2016-04-01,NC,HT,0,B,36,30,5,5,9,2,0,2,0,0,1,0,5,0,0,9,1,1,7,0.142857,7,1,2016,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,20160401HTNC0,2016-04-01,HT,NC,0,T,38,34,3,4,8,3,0,1,0,0,0,0,3,0,1,10,1,0,7,0.100000,10,1,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,20160401KTSK0,2016-04-01,SK,KT,0,B,36,36,4,4,10,5,0,1,0,0,0,0,0,0,0,7,1,1,5,0.375000,8,3,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,20200719LTSS0,2020-07-19,LT,SS,0,T,31,29,2,2,6,1,0,1,0,0,1,0,1,0,0,7,3,0,2,0.000000,2,0,2020,4,0.299663,0.314607,0.614270,0.228464,0.086142,0.263636,0.565217,0.154362,0.087248,0.288293,-14.656294,8.018131,0.010067,0.213500
636,20200719OBHT0,2020-07-19,HT,OB,0,B,38,32,4,4,8,0,1,0,0,0,0,0,6,0,0,5,1,0,7,0.571429,7,4,2020,7,0.340000,0.389513,0.729513,0.265918,0.123596,0.303167,0.681818,0.144737,0.095395,0.333920,-2.931033,20.276013,0.013158,0.250378
637,20200719OBHT0,2020-07-19,OB,HT,0,T,41,37,8,8,11,3,0,1,2,1,0,0,2,1,2,4,0,0,6,0.363636,11,4,2020,7,0.363934,0.433213,0.797147,0.314079,0.119134,0.360870,0.382979,0.152104,0.058252,0.362672,4.736070,28.247471,0.012945,0.272074
638,20200719WOSK0,2020-07-19,SK,WO,0,B,37,30,3,4,6,0,0,0,0,0,0,0,7,0,0,10,0,0,9,0.250000,8,2,2020,6,0.306931,0.365942,0.672873,0.246377,0.119565,0.287081,0.377049,0.200000,0.075410,0.307934,-9.804971,13.402075,0.026230,0.229604


In [18]:
team_hitter.columns = ['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '타자', '타수', '타점', '득점',
       '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비', '4구', '고4', '사구',
       '삼진', '병살타', '실책', '잔루', '득점권타율', '득점권타수', '득점권안타', '시즌', '단타', '상대출루율',
       '상대장타력', '상대ops', '상대타율', '상대ISO', '상대BABIP', '상대BB/K', '상대K%', '상대BB%', '상대wOBA', '상대wRAA',
       '상대wRC', '상대HR%', '상대GPA']

In [19]:
team_hitter

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타,시즌,단타,상대출루율,상대장타력,상대ops,상대타율,상대ISO,상대BABIP,상대BB/K,상대K%,상대BB%,상대wOBA,상대wRAA,상대wRC,상대HR%,상대GPA
0,20160401HHLG0,2016-04-01,LG,HH,0,B,47,42,4,5,9,2,0,1,2,1,1,0,4,0,0,11,0,0,8,0.333333,12,4,2016,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,20160401HHLG0,2016-04-01,HH,LG,0,T,52,46,4,4,13,2,0,0,0,0,3,0,3,0,0,10,1,2,12,0.200000,15,3,2016,11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,20160401HTNC0,2016-04-01,NC,HT,0,B,36,30,5,5,9,2,0,2,0,0,1,0,5,0,0,9,1,1,7,0.142857,7,1,2016,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,20160401HTNC0,2016-04-01,HT,NC,0,T,38,34,3,4,8,3,0,1,0,0,0,0,3,0,1,10,1,0,7,0.100000,10,1,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,20160401KTSK0,2016-04-01,SK,KT,0,B,36,36,4,4,10,5,0,1,0,0,0,0,0,0,0,7,1,1,5,0.375000,8,3,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,20200719LTSS0,2020-07-19,LT,SS,0,T,31,29,2,2,6,1,0,1,0,0,1,0,1,0,0,7,3,0,2,0.000000,2,0,2020,4,0.299663,0.314607,0.614270,0.228464,0.086142,0.263636,0.565217,0.154362,0.087248,0.288293,-14.656294,8.018131,0.010067,0.213500
636,20200719OBHT0,2020-07-19,HT,OB,0,B,38,32,4,4,8,0,1,0,0,0,0,0,6,0,0,5,1,0,7,0.571429,7,4,2020,7,0.340000,0.389513,0.729513,0.265918,0.123596,0.303167,0.681818,0.144737,0.095395,0.333920,-2.931033,20.276013,0.013158,0.250378
637,20200719OBHT0,2020-07-19,OB,HT,0,T,41,37,8,8,11,3,0,1,2,1,0,0,2,1,2,4,0,0,6,0.363636,11,4,2020,7,0.363934,0.433213,0.797147,0.314079,0.119134,0.360870,0.382979,0.152104,0.058252,0.362672,4.736070,28.247471,0.012945,0.272074
638,20200719WOSK0,2020-07-19,SK,WO,0,B,37,30,3,4,6,0,0,0,0,0,0,0,7,0,0,10,0,0,9,0.250000,8,2,2020,6,0.306931,0.365942,0.672873,0.246377,0.119565,0.287081,0.377049,0.200000,0.075410,0.307934,-9.804971,13.402075,0.026230,0.229604


In [20]:
team_hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA7-팀타자_상대별_세이머 매트릭스_합본.csv', encoding='utf-8-sig')

In [21]:
hitter_2016 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_개인타자_2016.csv', encoding='utf-8-sig')
hitter_2017 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_개인타자_2017.csv', encoding='utf-8-sig')
hitter_2018 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_개인타자_2018.csv', encoding='utf-8-sig')
hitter_2019 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_개인타자_2019.csv', encoding='utf-8-sig')
hitter_2020 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_개인타자_2020.csv', encoding='utf-8-sig')
hitter_list = [hitter_2016, hitter_2017, hitter_2018, hitter_2019, hitter_2020]
hitter = pd.concat(hitter_list)
hitter_list.append(hitter)

In [23]:
new_columns = "게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,선수코드,선발,타순,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타"
new_columns = new_columns.split(',')
for hitter_i in hitter_list:
    hitter_i.columns = new_columns

In [27]:
for hitter_i in hitter_list:
  hitter_i['일자'] = pd.to_datetime(hitter_i['일자'],format='%Y%m%d', errors='raise')
  hitter_i['시즌'] = hitter_i['일자'].dt.year 

for hitter_i in hitter_list:
  hitter_i['단타'] = hitter_i['안타'] - (hitter_i['2루타'] + hitter_i['3루타'] + hitter_i['홈런'])

In [28]:
hitter

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,선수코드,선발,타순,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타,시즌,단타
0,20160401HHLG0,2016-04-01,HH,LG,0,T,60404,0,3,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0,0,2016,1
1,20160401HHLG0,2016-04-01,LG,HH,0,B,61102,1,8,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0.0,0,0,2016,0
2,20160401HHLG0,2016-04-01,LG,HH,0,B,61186,1,7,4,4,2,2,3,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1.0,1,1,2016,2
3,20160401HHLG0,2016-04-01,LG,HH,0,B,62164,0,9,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0,0,2016,0
4,20160401HHLG0,2016-04-01,HH,LG,0,T,62700,1,9,2,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0,0,2016,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8055,20200719WOSK0,2020-07-19,SK,WO,0,B,75847,1,3,4,4,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0.0,1,0,2020,1
8056,20200719WOSK0,2020-07-19,SK,WO,0,B,76802,1,9,4,3,1,0,3,0,0,0,0,0,0,0,1,0,0,0,0,0,4,1.0,1,1,2020,3
8057,20200719WOSK0,2020-07-19,SK,WO,0,B,77463,0,6,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0,0,2020,0
8058,20200719WOSK0,2020-07-19,WO,SK,0,T,78168,1,1,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,1,0,2020,0


In [30]:
def runBaseP(row):
    previous = hitter[(hitter['시즌'] == row['시즌']) & (hitter['선수코드'] == row['선수코드']) & (hitter['상대팀코드'] == row['상대팀코드']) & (hitter['일자'] < row['일자']) ]
    if len(previous) == 0:
        row['출루율'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['안타'] + previous_sum['4구']  + previous_sum['사구']
    down = previous_sum['사구'] + previous_sum['타수'] + previous_sum['4구'] + previous_sum['희비']
    if down == 0:
        row['출루율'] = 0
        return row
    row['출루율'] = up / down
    return row

def longHitP(row):
    previous = hitter[(hitter['시즌'] == row['시즌']) & (hitter['선수코드'] == row['선수코드']) & (hitter['상대팀코드'] == row['상대팀코드']) & (hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['장타력'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['단타'] * 1 + previous_sum['2루타'] * 2 + previous_sum['3루타'] * 3 + previous_sum['홈런'] * 4
    down = previous_sum['타수']
    if down == 0:
        row['장타력'] = 0
        return row
    row['장타력'] = up / down

    return row
def hitP(row):
    previous = hitter[(hitter['시즌'] == row['시즌']) & (hitter['선수코드'] == row['선수코드']) & (hitter['상대팀코드'] == row['상대팀코드']) & (hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['타율'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['안타']
    down = previous_sum['타수']
    if down == 0:
        row['타율'] = 0
        return row
    row['타율'] = up / down
    return row

def createBABIP(row):
  previous = hitter[(hitter['시즌'] == row['시즌']) & (hitter['선수코드'] == row['선수코드']) & (hitter['상대팀코드'] == row['상대팀코드']) & (hitter['일자'] < row['일자'])]
  if len(previous) == 0:
      row['BABIP'] = 0
      return row
  previous_sum = previous.sum()
  up = previous_sum['안타'] - previous_sum['홈런']
  down = previous_sum['타수'] - previous_sum['홈런'] - previous_sum['삼진'] + previous_sum['희비']
  if down == 0:
      row['BABIP'] = 0
      return row
  row['BABIP'] = up / down
  return row

def createBBK(row):
    previous = hitter[(hitter['시즌'] == row['시즌']) & (hitter['선수코드'] == row['선수코드']) & (hitter['상대팀코드'] == row['상대팀코드']) & (hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['BB/K'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['4구']
    down = previous_sum['삼진']
    if down == 0:
        row['BB/K'] = 0
        return row
    row['BB/K'] = up / down
    return row

def createK_BB(row):
    previous = hitter[(hitter['시즌'] == row['시즌']) & (hitter['선수코드'] == row['선수코드']) & (hitter['상대팀코드'] == row['상대팀코드']) & (hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['K%'] = 0
        row['BB%'] = 0
        return row
    previous_sum = previous.sum()
    up_k = previous_sum['삼진']
    up_BB = previous_sum['4구'] - previous_sum['고4']
    down = previous_sum['타자'] - previous_sum['고4']
    if down == 0:
        row['K%'] = 0
        row['BB%'] = 0
        return row
    row['K%'] = up_k / down
    row['BB%'] = up_BB / down
    return row

In [37]:
from tqdm import tqdm 
tqdm.pandas()

In [ ]:
hitter = hitter.apply(runBaseP,axis=1)
hitter = hitter.apply(longHitP,axis=1)
hitter['ops'] = hitter['출루율'] + hitter['장타력']
hitter = hitter.apply(hitP,axis=1)
hitter['ISO'] = hitter['장타력'] - hitter['타율']
hitter = hitter.apply(createBABIP,axis=1)
hitter = hitter.apply(createBBK,axis=1)
hitter = hitter.apply(createK_BB,axis=1)

In [ ]:
def createwOBA(row):
    previous = hitter[(hitter['시즌'] == row['시즌']) & (hitter['선수코드'] == row['선수코드'])& (hitter['상대팀코드'] == row['상대팀코드']) & (hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['wOBA'] = 0
        return row
    previous_sum = previous.sum()
    ct = c_dic[row['시즌']]
    up = (ct[0] * (previous_sum['4구'] - previous_sum['고4']) + ct[1] * previous_sum['사구'] + ct[2] * previous_sum['단타'] + ct[3] * previous_sum['2루타'] + ct[4] * previous_sum['3루타'] + ct[5] * previous_sum['홈런'] )
    down = (previous_sum['타수'] + previous_sum['4구'] - previous_sum['고4'] + previous_sum['사구'] + previous_sum['희비'])
    if down == 0:
        row['wOBA'] = 0
        return row
    row['wOBA'] = up / down
    return row

def createwRAA(row):
    previous = hitter[(hitter['시즌'] == row['시즌']) & (hitter['선수코드'] == row['선수코드'])& (hitter['상대팀코드'] == row['상대팀코드']) & (hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['wRAA'] = 0
        return row
    previous_sum = previous.sum()
    ct = c_dic2[row['시즌']]
    up = row['wOBA'] - ct[0]
    down = ct[1]
    row['wRAA'] = up / down * previous_sum['타자']
    return row

def createwRC(row):
    previous = hitter[(hitter['시즌'] == row['시즌']) & (hitter['선수코드'] == row['선수코드'])& (hitter['상대팀코드'] == row['상대팀코드']) & (hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['wRC'] = 0
        return row
    previous_sum = previous.sum()
    ct = c_dic2[row['시즌']]
    up = ct[2]
    down = ct[3]
    row['wRC'] = (up / down) * previous_sum['타자'] + row['wRAA']
    return row

def createHR(row):
    previous = hitter[(hitter['시즌'] == row['시즌']) & (hitter['선수코드'] == row['선수코드'])& (hitter['상대팀코드'] == row['상대팀코드']) & (hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['HR%'] = 0
        return row
    previous_sum = previous.sum()
    up_hr = previous_sum['홈런']
    down = previous_sum['타자'] - previous_sum['고4']
    if down == 0:
        row['HR%'] = 0
        return row
        
    row['HR%'] = up_hr / down
    return row

In [ ]:
hitter = hitter.apply(createwOBA, axis=1)
hitter = hitter.apply(createwRAA, axis=1)
hitter = hitter.apply(createwRC, axis=1)
hitter = hitter.apply(createHR, axis=1)
hitter['GPA'] = (1.8 * hitter['출루율'] + hitter['장타력']) / 4 

In [ ]:
hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA7-개인타자_상대별_세이머 매트릭스_합본.csv', encoding='utf-8-sig')